## <center>Task 1 : Don't patronize me</center>

In [524]:
import pandas as pd
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from dont_patronize_me import DontPatronizeMe
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string
import re
import nltk
from nltk import word_tokenize
from collections import Counter

# Imblearn methods
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTEN, ADASYN, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE

# Text augmentation techniques
from textaugment import MIXUP

import seaborn as sns
import matplotlib.pyplot as plt

from keras.preprocessing.text import Tokenizer
import matplotlib.pyplot as plt
#https://github.com/learn-co-students/dsc-classification-with-word-embeddings-codealong-online-ds-pt-100719
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow import keras
from keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from self_paced_ensemble import SelfPacedEnsembleClassifier
from self_paced_ensemble.self_paced_ensemble.base import sort_dict_by_key
from self_paced_ensemble.utils._plot import plot_2Dprojection_and_cardinality
from sklearn.tree import DecisionTreeClassifier

# Word2vec
import gensim

In [428]:
dpm = DontPatronizeMe('.')
# This method loads the subtask 1 data
dpm.load_task1()
# which we can then access as a dataframe
dataset = dpm.train_task1_df
dataset.head()

,par_id,art_id,keyword,country,text,label,orig_label
0,1,@@24942188,hopeless,ph,"we 're living in times of absolute insanity , ...",0,0
1,2,@@21968160,migrant,gh,"in libya today , there are countless number of...",0,0
2,3,@@16584954,immigrant,ie,"""white house press secretary sean spicer said ...",0,0
3,4,@@7811231,disabled,nz,council customers only signs would be displaye...,0,0
4,5,@@1494111,refugee,ca,""""""" just like we received migrants fleeing el ...",0,0


In [429]:
train = pd.read_csv("train_subset.csv")
valid = pd.read_csv("validation_subset.csv")

In [430]:
print("Class 0 - Class 1: {} - {}".format(len(train[train.label == 0]), len(train[train.label == 1])))

Class 0 - Class 1: 7581 - 794


## Preprocessing

In [431]:
def text_preprocessing(text, lemmatize):
    if not isinstance(text, str):
        text = text.decode('ISO-8859-1')
    
    text = re.sub('[^a-zA-Z]', ' ', text) # Clear the special characters from our dataset
    text = text.lower() 
    text = text.split() 
    text = ' '.join(text)
    #print('\tClear Text.\n', text)

    # Tokenize
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #print('\tTokenizing.\n', tokens)

    # Removing the stopwords
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in set(stop)]
    #print('\tRemoving the stopwords.\n', tokens)
    
    # Remove words shorter than 3 characters
    tokens = [token for token in tokens if len(token) >= 2]
    #print('\tRemoving the words shorter than 3 characters\n', tokens)
    

    if lemmatize:
        # Lemmatize
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        #print('\tLemmatizing.\n', tokens)
    else:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    
    # Preprocessed text
    tokens = ' '.join(tokens)
    
    return tokens


train['cleaned_lemma'] = [text_preprocessing(doc, True) for doc in train.text]
valid['cleaned_lemma'] = [text_preprocessing(str(doc), True) for doc in valid.text]

train['cleaned_stem'] = [text_preprocessing(doc, False) for doc in train.text]
valid['cleaned_stem'] = [text_preprocessing(str(doc), False) for doc in valid.text]

In [434]:
vectorizer_tfidf = TfidfVectorizer(max_features = 2000)

X_train_lemma = vectorizer_tfidf.fit_transform(train.cleaned_lemma)
X_valid_lemma = vectorizer_tfidf.transform(valid.cleaned_lemma)

X_train_stem = vectorizer_tfidf.fit_transform(train.cleaned_stem)
X_valid_stem = vectorizer_tfidf.transform(valid.cleaned_stem)


# Balancing the data
X_train_lemma_SMOTE, y_train_lemma_SMOTE = SMOTE(random_state = 42).fit_resample(X_train_lemma, train.label)
X_train_lemma_ROS, y_train_lemma_ROS = RandomOverSampler(random_state = 42).fit_resample(X_train_lemma, train.label)
X_train_lemma_ADASYN, y_train_lemma_ADASYN = ADASYN(random_state = 42).fit_resample(X_train_lemma, train.label)
X_train_lemma_BorderlineSMOTE, y_train_lemma_BorderlineSMOTE = BorderlineSMOTE(random_state = 42).fit_resample(X_train_lemma, train.label)
X_train_lemma_SVMSMOTE, y_train_lemma_SVMSMOTE = SVMSMOTE(random_state = 42).fit_resample(X_train_lemma, train.label)

# Balancing the data
X_train_stem_SMOTE, y_train_stem_SMOTE = SMOTE(random_state = 42).fit_resample(X_train_stem, train.label)
X_train_stem_ROS, y_train_stem_ROS = RandomOverSampler(random_state = 42).fit_resample(X_train_stem, train.label)
X_train_stem_ADASYN, y_train_stem_ADASYN = ADASYN(random_state = 42).fit_resample(X_train_stem, train.label)
X_train_stem_BorderlineSMOTE, y_train_stem_BorderlineSMOTE = BorderlineSMOTE(random_state = 42).fit_resample(X_train_stem, train.label)
X_train_stem_SVMSMOTE, y_train_stem_SVMSMOTE = SVMSMOTE(random_state = 42).fit_resample(X_train_stem, train.label)

### Deep Learning model for dataset unbalanced

In [496]:
def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0
    
    return class_weight

def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

class_weights = create_class_weight({0: 7581, 1: 794})

- X_train_lemmatized

In [470]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma.toarray(), train.label, epochs = 20, class_weight = class_weights)

Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_228 (Dense)           (None, 32)                64032     
                                                                 
 dense_229 (Dense)           (None, 512)               16896     
                                                                 
 dense_230 (Dense)           (None, 256)               131328    
                                                                 
 dense_231 (Dense)           (None, 128)               32896     
                                                                 
 dense_232 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
262/262 [===========================

In [472]:
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
y_pred

f1_score(y_pred, valid.label)

0.2727272727272727

- X_train_stem

In [473]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_stem.toarray(), train.label, epochs = 20, class_weight = class_weights)

Model: "sequential_65"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_233 (Dense)           (None, 32)                64032     
                                                                 
 dense_234 (Dense)           (None, 512)               16896     
                                                                 
 dense_235 (Dense)           (None, 256)               131328    
                                                                 
 dense_236 (Dense)           (None, 128)               32896     
                                                                 
 dense_237 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
262/262 [===========================

In [474]:
y = model.predict(X_valid_stem.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.2698412698412698

### Deep Learning model for oversampled dataset

- Lemmatized oversampled datasets

In [475]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma_SMOTE.toarray(), y_train_lemma_SMOTE, epochs = 20)

Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_238 (Dense)           (None, 32)                64032     
                                                                 
 dense_239 (Dense)           (None, 512)               16896     
                                                                 
 dense_240 (Dense)           (None, 256)               131328    
                                                                 
 dense_241 (Dense)           (None, 128)               32896     
                                                                 
 dense_242 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [476]:
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.28228228228228225

In [477]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma_ROS.toarray(), y_train_lemma_ROS, epochs = 20)

Model: "sequential_67"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_243 (Dense)           (None, 32)                64032     
                                                                 
 dense_244 (Dense)           (None, 512)               16896     
                                                                 
 dense_245 (Dense)           (None, 256)               131328    
                                                                 
 dense_246 (Dense)           (None, 128)               32896     
                                                                 
 dense_247 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [478]:
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.21453287197231835

In [479]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma_ADASYN.toarray(), y_train_lemma_ADASYN, epochs = 20)

Model: "sequential_68"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_248 (Dense)           (None, 32)                64032     
                                                                 
 dense_249 (Dense)           (None, 512)               16896     
                                                                 
 dense_250 (Dense)           (None, 256)               131328    
                                                                 
 dense_251 (Dense)           (None, 128)               32896     
                                                                 
 dense_252 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
467/467 [===========================

In [480]:
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.2774566473988439

In [481]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma_BorderlineSMOTE.toarray(), y_train_lemma_BorderlineSMOTE, epochs = 20)

Model: "sequential_69"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_253 (Dense)           (None, 32)                64032     
                                                                 
 dense_254 (Dense)           (None, 512)               16896     
                                                                 
 dense_255 (Dense)           (None, 256)               131328    
                                                                 
 dense_256 (Dense)           (None, 128)               32896     
                                                                 
 dense_257 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [482]:
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.19292604501607716

In [483]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma_SVMSMOTE.toarray(), y_train_lemma_SVMSMOTE, epochs = 20)

Model: "sequential_70"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_258 (Dense)           (None, 32)                64032     
                                                                 
 dense_259 (Dense)           (None, 512)               16896     
                                                                 
 dense_260 (Dense)           (None, 256)               131328    
                                                                 
 dense_261 (Dense)           (None, 128)               32896     
                                                                 
 dense_262 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [484]:
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.3187660668380463

- Stemmed oversampled datasets

In [485]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_stem_SMOTE.toarray(), y_train_stem_SMOTE, epochs = 20)

Model: "sequential_71"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_263 (Dense)           (None, 32)                64032     
                                                                 
 dense_264 (Dense)           (None, 512)               16896     
                                                                 
 dense_265 (Dense)           (None, 256)               131328    
                                                                 
 dense_266 (Dense)           (None, 128)               32896     
                                                                 
 dense_267 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [486]:
y = model.predict(X_valid_stem.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.28901734104046245

In [487]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_stem_ROS.toarray(), y_train_stem_ROS, epochs = 20)

Model: "sequential_72"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_268 (Dense)           (None, 32)                64032     
                                                                 
 dense_269 (Dense)           (None, 512)               16896     
                                                                 
 dense_270 (Dense)           (None, 256)               131328    
                                                                 
 dense_271 (Dense)           (None, 128)               32896     
                                                                 
 dense_272 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [488]:
y = model.predict(X_valid_stem.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.2922077922077922

In [490]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_stem_ADASYN.toarray(), y_train_stem_ADASYN, epochs = 20)

Model: "sequential_74"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_278 (Dense)           (None, 32)                64032     
                                                                 
 dense_279 (Dense)           (None, 512)               16896     
                                                                 
 dense_280 (Dense)           (None, 256)               131328    
                                                                 
 dense_281 (Dense)           (None, 128)               32896     
                                                                 
 dense_282 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [491]:
y = model.predict(X_valid_stem.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.2821316614420063

In [492]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_stem_BorderlineSMOTE.toarray(), y_train_stem_BorderlineSMOTE, epochs = 20)

Model: "sequential_75"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_283 (Dense)           (None, 32)                64032     
                                                                 
 dense_284 (Dense)           (None, 512)               16896     
                                                                 
 dense_285 (Dense)           (None, 256)               131328    
                                                                 
 dense_286 (Dense)           (None, 128)               32896     
                                                                 
 dense_287 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [493]:
y = model.predict(X_valid_stem.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.28648648648648645

In [494]:
model = Sequential([
Dense(input_dim = 2000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_stem_SVMSMOTE.toarray(), y_train_stem_SVMSMOTE, epochs = 20)

Model: "sequential_76"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_288 (Dense)           (None, 32)                64032     
                                                                 
 dense_289 (Dense)           (None, 512)               16896     
                                                                 
 dense_290 (Dense)           (None, 256)               131328    
                                                                 
 dense_291 (Dense)           (None, 128)               32896     
                                                                 
 dense_292 (Dense)           (None, 1)                 129       
                                                                 
Total params: 245,281
Trainable params: 245,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
474/474 [===========================

In [495]:
y = model.predict(X_valid_stem.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

0.31666666666666665

## Tokenizer and pad sequence, followed by LSTM

In [508]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train.cleaned_lemma))
#try texts_to_matrix
list_tokenized_train = tokenizer.texts_to_sequences(train.cleaned_lemma)
list_tokenized_test = tokenizer.texts_to_sequences(valid.cleaned_lemma)

maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

rnn_tiny = Sequential()
embedding_size = 128

rnn_tiny.add(Embedding(max_features, embedding_size))
#adding LSTM layer to help 'forget' then pooling
rnn_tiny.add(layers.LSTM(128, return_sequences=True,name='lstm_layer'))        
rnn_tiny.add(layers.GlobalMaxPool1D()) 
rnn_tiny.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l1(.0001) ))
rnn_tiny.add(Dropout(0.1))
rnn_tiny.add(Dense(1, activation='sigmoid'))

rnn_tiny.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy', get_f1])
history_tiny_rnn = rnn_tiny.fit(X_t, train.label, epochs=10)

y = rnn_tiny.predict(X_te)
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

Epoch 1/10
262/262 [==============================] - 15s 52ms/step - loss: 0.3616 - accuracy: 0.9019 - get_f1: 4.4903e-04
Epoch 2/10
262/262 [==============================] - 14s 52ms/step - loss: 0.2661 - accuracy: 0.9079 - get_f1: 0.1214
Epoch 3/10
262/262 [==============================] - 14s 54ms/step - loss: 0.2012 - accuracy: 0.9280 - get_f1: 0.4351
Epoch 4/10
262/262 [==============================] - 14s 53ms/step - loss: 0.1600 - accuracy: 0.9442 - get_f1: 0.6029
Epoch 5/10
262/262 [==============================] - 14s 55ms/step - loss: 0.1263 - accuracy: 0.9587 - get_f1: 0.6941
Epoch 6/10
262/262 [==============================] - 14s 55ms/step - loss: 0.0945 - accuracy: 0.9717 - get_f1: 0.7663
Epoch 7/10
262/262 [==============================] - 14s 55ms/step - loss: 0.0659 - accuracy: 0.9820 - get_f1: 0.8634
Epoch 8/10
262/262 [==============================] - 15s 56ms/step - loss: 0.0493 - accuracy: 0.9883 - get_f1: 0.8826
Epoch 9/10
262/262 [========================

0.23170731707317074

In [509]:
max_features = 2000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train.cleaned_stem))
#try texts_to_matrix
list_tokenized_train = tokenizer.texts_to_sequences(train.cleaned_stem)
list_tokenized_test = tokenizer.texts_to_sequences(valid.cleaned_stem)

maxlen = 100
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)

rnn_tiny = Sequential()
embedding_size = 128

rnn_tiny.add(Embedding(max_features, embedding_size))
#adding LSTM layer to help 'forget' then pooling
rnn_tiny.add(layers.LSTM(128, return_sequences=True,name='lstm_layer'))        
rnn_tiny.add(layers.GlobalMaxPool1D()) 
rnn_tiny.add(Dense(64, activation='relu',kernel_regularizer=regularizers.l1(.0001) ))
rnn_tiny.add(Dropout(0.1))
rnn_tiny.add(Dense(1, activation='sigmoid'))

rnn_tiny.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy', get_f1])
history_tiny_rnn = rnn_tiny.fit(X_t, train.label, epochs=10)

y = rnn_tiny.predict(X_te)
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

Epoch 1/10
262/262 [==============================] - 17s 55ms/step - loss: 0.3548 - accuracy: 0.9052 - get_f1: 0.0000e+00
Epoch 2/10
262/262 [==============================] - 15s 56ms/step - loss: 0.2482 - accuracy: 0.9151 - get_f1: 0.2248
Epoch 3/10
262/262 [==============================] - 15s 56ms/step - loss: 0.1897 - accuracy: 0.9321 - get_f1: 0.4917
Epoch 4/10
262/262 [==============================] - 15s 57ms/step - loss: 0.1457 - accuracy: 0.9503 - get_f1: 0.6494
Epoch 5/10
262/262 [==============================] - 15s 58ms/step - loss: 0.1073 - accuracy: 0.9655 - get_f1: 0.7447
Epoch 6/10
262/262 [==============================] - 15s 57ms/step - loss: 0.0811 - accuracy: 0.9764 - get_f1: 0.8309
Epoch 7/10
262/262 [==============================] - 15s 58ms/step - loss: 0.0552 - accuracy: 0.9869 - get_f1: 0.8799
Epoch 8/10
262/262 [==============================] - 16s 59ms/step - loss: 0.0411 - accuracy: 0.9904 - get_f1: 0.9122
Epoch 9/10
262/262 [========================

0.2789473684210526

## Datasets applied on ML algorithms

In [520]:
# Lemmatized dataset

lr = LogisticRegression(class_weight = 'balanced')
rf = RandomForestClassifier(class_weight = 'balanced')
svm = SVC()
xgb = XGBClassifier()

lr.fit(X_train_lemma, train.label)
rf.fit(X_train_lemma, train.label)
svm.fit(X_train_lemma, train.label)
xgb.fit(X_train_lemma, train.label)

print("F1 score for Logistic Regression on train is ", f1_score(lr.predict(X_train_lemma), train.label))
print("F1 score for Random Forest on train is ", f1_score(rf.predict(X_train_lemma), train.label))
print("F1 score for SVM on train is ", f1_score(svm.predict(X_train_lemma), train.label))
print("F1 score for XGB on train is ", f1_score(xgb.predict(X_train_lemma), train.label))
print()
print("F1 score for Logistic Regression on validation is ", f1_score(lr.predict(X_valid_lemma), valid.label))
print("F1 score for Random Forest on validation is ", f1_score(rf.predict(X_valid_lemma), valid.label))
print("F1 score for SVM on validation is ", f1_score(svm.predict(X_valid_lemma), valid.label))
print("F1 score for XGB on validation is ", f1_score(xgb.predict(X_valid_lemma), valid.label))

print("----------------")

# Stemmed dataset
lr = LogisticRegression(class_weight = 'balanced')
rf = RandomForestClassifier()
svm = SVC(class_weight = 'balanced')
xgb = XGBClassifier()

lr.fit(X_train_stem, train.label)
rf.fit(X_train_stem, train.label)
svm.fit(X_train_stem, train.label)
xgb.fit(X_train_stem, train.label)

print("F1 score for Logistic Regression on train is ", f1_score(lr.predict(X_train_stem), train.label))
print("F1 score for Random Forest on train is ", f1_score(rf.predict(X_train_stem), train.label))
print("F1 score for SVM on train is ", f1_score(svm.predict(X_train_stem), train.label))
print("F1 score for XGB on train is ", f1_score(xgb.predict(X_train_stem), train.label))
print()
print("F1 score for Logistic Regression on validation is ", f1_score(lr.predict(X_valid_stem), valid.label))
print("F1 score for Random Forest on validation is ", f1_score(rf.predict(X_valid_stem), valid.label))
print("F1 score for SVM on validation is ", f1_score(svm.predict(X_valid_stem), valid.label))
print("F1 score for XGB on validation is ", f1_score(xgb.predict(X_valid_stem), valid.label))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:29:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 score for Logistic Regression on train is  0.5899960614415124
F1 score for Random Forest on train is  1.0
F1 score for SVM on train is  0.599647266313933
F1 score for XGB on train is  0.6633081444164567

F1 score for Logistic Regression on validation is  0.34076433121019106
F1 score for Random Forest on validation is  0.057971014492753624
F1 score for SVM on validation is  0.057971014492753624
F1 score for XGB on validation is  0.15510204081632653
----------------


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:29:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 score for Logistic Regression on train is  0.6011976047904192
F1 score for Random Forest on train is  0.9993698802772526
F1 score for SVM on train is  0.9808523780111179
F1 score for XGB on train is  0.7047308319738989

F1 score for Logistic Regression on validation is  0.35197368421052627
F1 score for Random Forest on validation is  0.01990049751243781
F1 score for SVM on validation is  0.2756183745583039
F1 score for XGB on validation is  0.1721311475409836


In [522]:
# Lemmatized dataset

lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
xgb = XGBClassifier()

lr.fit(X_train_lemma_SMOTE, y_train_lemma_SMOTE)
rf.fit(X_train_lemma_SMOTE, y_train_lemma_SMOTE)
svm.fit(X_train_lemma_SMOTE, y_train_lemma_SMOTE)
xgb.fit(X_train_lemma_SMOTE, y_train_lemma_SMOTE)

print("F1 score for Logistic Regression on train is ", f1_score(lr.predict(X_train_lemma_SMOTE), y_train_lemma_SMOTE))
print("F1 score for Random Forest on train is ", f1_score(rf.predict(X_train_lemma_SMOTE), y_train_lemma_SMOTE))
print("F1 score for SVM on train is ", f1_score(svm.predict(X_train_lemma_SMOTE), y_train_lemma_SMOTE))
print("F1 score for XGB on train is ", f1_score(xgb.predict(X_train_lemma_SMOTE), y_train_lemma_SMOTE))
print()
print("F1 score for Logistic Regression on validation is ", f1_score(lr.predict(X_valid_lemma), valid.label))
print("F1 score for Random Forest on validation is ", f1_score(rf.predict(X_valid_lemma), valid.label))
print("F1 score for SVM on validation is ", f1_score(svm.predict(X_valid_lemma), valid.label))
print("F1 score for XGB on validation is ", f1_score(xgb.predict(X_valid_lemma), valid.label))

print("----------------")

# Stemmed dataset
lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
xgb = XGBClassifier()

lr.fit(X_train_lemma_SVMSMOTE, y_train_lemma_SVMSMOTE)
rf.fit(X_train_lemma_SVMSMOTE, y_train_lemma_SVMSMOTE)
svm.fit(X_train_lemma_SVMSMOTE, y_train_lemma_SVMSMOTE)
xgb.fit(X_train_lemma_SVMSMOTE, y_train_lemma_SVMSMOTE)

print("F1 score for Logistic Regression on train is ", f1_score(lr.predict(X_train_lemma_SVMSMOTE), y_train_lemma_SVMSMOTE))
print("F1 score for Random Forest on train is ", f1_score(rf.predict(X_train_lemma_SVMSMOTE), y_train_lemma_SVMSMOTE))
print("F1 score for SVM on train is ", f1_score(svm.predict(X_train_lemma_SVMSMOTE), y_train_lemma_SVMSMOTE))
print("F1 score for XGB on train is ", f1_score(xgb.predict(X_train_lemma_SVMSMOTE), y_train_lemma_SVMSMOTE))
print()
print("F1 score for Logistic Regression on validation is ", f1_score(lr.predict(X_valid_lemma), valid.label))
print("F1 score for Random Forest on validation is ", f1_score(rf.predict(X_valid_lemma), valid.label))
print("F1 score for SVM on validation is ", f1_score(svm.predict(X_valid_lemma), valid.label))
print("F1 score for XGB on validation is ", f1_score(xgb.predict(X_valid_lemma), valid.label))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:36:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 score for Logistic Regression on train is  0.9276644863409004
F1 score for Random Forest on train is  1.0
F1 score for SVM on train is  0.9978187586753916
F1 score for XGB on train is  0.9716719626800081

F1 score for Logistic Regression on validation is  0.34676007005253934
F1 score for Random Forest on validation is  0.20610687022900764
F1 score for SVM on validation is  0.10138248847926268
F1 score for XGB on validation is  0.23225806451612904
----------------


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:37:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 score for Logistic Regression on train is  0.9316472545757072
F1 score for Random Forest on train is  1.0
F1 score for SVM on train is  0.9930942895086322
F1 score for XGB on train is  0.9713319810682893

F1 score for Logistic Regression on validation is  0.350561797752809
F1 score for Random Forest on validation is  0.15833333333333333
F1 score for SVM on validation is  0.13392857142857142
F1 score for XGB on validation is  0.23948220064724918


In [523]:
# Lemmatized dataset

lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
xgb = XGBClassifier()

lr.fit(X_train_stem_SMOTE, y_train_stem_SMOTE)
rf.fit(X_train_stem_SMOTE, y_train_stem_SMOTE)
svm.fit(X_train_stem_SMOTE, y_train_stem_SMOTE)
xgb.fit(X_train_stem_SMOTE, y_train_stem_SMOTE)

print("F1 score for Logistic Regression on train is ", f1_score(lr.predict(X_train_stem_SMOTE), y_train_stem_SMOTE))
print("F1 score for Random Forest on train is ", f1_score(rf.predict(X_train_stem_SMOTE), y_train_stem_SMOTE))
print("F1 score for SVM on train is ", f1_score(svm.predict(X_train_stem_SMOTE), y_train_stem_SMOTE))
print("F1 score for XGB on train is ", f1_score(xgb.predict(X_train_stem_SMOTE), y_train_stem_SMOTE))
print()
print("F1 score for Logistic Regression on validation is ", f1_score(lr.predict(X_valid_stem), valid.label))
print("F1 score for Random Forest on validation is ", f1_score(rf.predict(X_valid_stem), valid.label))
print("F1 score for SVM on validation is ", f1_score(svm.predict(X_valid_stem), valid.label))
print("F1 score for XGB on validation is ", f1_score(xgb.predict(X_valid_stem), valid.label))

print("----------------")

# Stemmed dataset
lr = LogisticRegression()
rf = RandomForestClassifier()
svm = SVC()
xgb = XGBClassifier()

lr.fit(X_train_stem_SVMSMOTE, y_train_stem_SVMSMOTE)
rf.fit(X_train_stem_SVMSMOTE, y_train_stem_SVMSMOTE)
svm.fit(X_train_stem_SVMSMOTE, y_train_stem_SVMSMOTE)
xgb.fit(X_train_stem_SVMSMOTE, y_train_stem_SVMSMOTE)

print("F1 score for Logistic Regression on train is ", f1_score(lr.predict(X_train_stem_SVMSMOTE), y_train_stem_SVMSMOTE))
print("F1 score for Random Forest on train is ", f1_score(rf.predict(X_train_stem_SVMSMOTE), y_train_stem_SVMSMOTE))
print("F1 score for SVM on train is ", f1_score(svm.predict(X_train_stem_SVMSMOTE), y_train_stem_SVMSMOTE))
print("F1 score for XGB on train is ", f1_score(xgb.predict(X_train_stem_SVMSMOTE), y_train_stem_SVMSMOTE))
print()
print("F1 score for Logistic Regression on validation is ", f1_score(lr.predict(X_valid_stem), valid.label))
print("F1 score for Random Forest on validation is ", f1_score(rf.predict(X_valid_stem), valid.label))
print("F1 score for SVM on validation is ", f1_score(svm.predict(X_valid_stem), valid.label))
print("F1 score for XGB on validation is ", f1_score(xgb.predict(X_valid_stem), valid.label))

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:40:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 score for Logistic Regression on train is  0.9310211946050096
F1 score for Random Forest on train is  1.0
F1 score for SVM on train is  0.9982160555004956
F1 score for XGB on train is  0.9723197407507427

F1 score for Logistic Regression on validation is  0.38007380073800734
F1 score for Random Forest on validation is  0.2007722007722008
F1 score for SVM on validation is  0.13636363636363635
F1 score for XGB on validation is  0.225705329153605
----------------


C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[01:41:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
F1 score for Logistic Regression on train is  0.9418673705047839
F1 score for Random Forest on train is  1.0
F1 score for SVM on train is  0.9946290033817387
F1 score for XGB on train is  0.9728598433702403

F1 score for Logistic Regression on validation is  0.3744493392070484
F1 score for Random Forest on validation is  0.12605042016806722
F1 score for SVM on validation is  0.19742489270386268
F1 score for XGB on validation is  0.2382445141065831


## Word2Vec

In [545]:
documents_lemma = [_text.split() for _text in train.cleaned_lemma]

w2v_model = gensim.models.word2vec.Word2Vec(vector_size = 300, 
                                            window = 2, 
                                            min_count = 2)
w2v_model.build_vocab(documents_lemma)
words = w2v_model.wv.index_to_key
vocab_size = len(words)
print("Vocab size", vocab_size)
w2v_model.train(documents_lemma, total_examples=len(documents_lemma), epochs = 50)

Vocab size 11815


(9185609, 10171650)

In [546]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train.cleaned_lemma)

vocab_size = len(tokenizer.word_index) + 1
print("Total words", vocab_size)

x_train = pad_sequences(tokenizer.texts_to_sequences(train.cleaned_lemma), maxlen = 300)
x_test = pad_sequences(tokenizer.texts_to_sequences(valid.cleaned_lemma), maxlen = 300)

Total words 23228


In [548]:
embedding_matrix = np.zeros((vocab_size, 300))
for word, i in tokenizer.word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=300, trainable=False)

(23228, 300)


In [553]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=[get_f1])
history = model.fit(x_train, train.label,
                    batch_size=32,
                    epochs=20,
                    verbose=1)

y = model.predict(x_test)
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

Model: "sequential_89"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_37 (Embedding)    (None, 300, 300)          6968400   
                                                                 
 lstm_9 (LSTM)               (None, 128)               219648    
                                                                 
 dense_313 (Dense)           (None, 1)                 129       
                                                                 
Total params: 7,188,177
Trainable params: 219,777
Non-trainable params: 6,968,400
_________________________________________________________________
Epoch 1/20
262/262 [==============================] - 42s 154ms/step - loss: 0.2920 - get_f1: 0.0248
Epoch 2/20
262/262 [==============================] - 45s 171ms/step - loss: 0.2495 - get_f1: 0.1290
Epoch 3/20
262/262 [==============================] - 42s 159ms/step - loss: 0.2275 - get_f1: 0.234

0.23076923076923075

### Self paced ensemble

In [560]:
# Train an SPE classifier
clf = SelfPacedEnsembleClassifier(
        base_estimator=RandomForestClassifier(), 
        n_estimators=10,
    ).fit(X_train_lemma, train.label)

# Predict with an SPE classifier
y_pred = clf.predict(X_valid_lemma)
print(f1_score(clf.predict(X_train_lemma), train.label))
print(f1_score(y_pred, valid.label))

0.43447332421340623
0.3132036847492323


In [562]:
# Train an SPE classifier
clf = SelfPacedEnsembleClassifier(
        base_estimator=RandomForestClassifier(), 
        n_estimators=10,
    ).fit(X_train_stem, train.label)

# Predict with an SPE classifier
y_pred = clf.predict(X_valid_stem)
print(f1_score(clf.predict(X_train_stem), train.label))
print(f1_score(y_pred, valid.label))

0.41386499869689863
0.30931796349663787


# Data Augmentation

In [563]:
hate1 = pd.read_csv("hate1.csv")
hate1 = hate1[hate1.label == 1]
hate1 = hate1.rename({'tweet': 'text'}, axis = 1)

hate2 = pd.read_excel("hate2.xlsx")
hate2 = hate2[hate2.Label == 1]
hate2 = hate2.rename({'Sentences': 'text', 'Label': 'label'}, axis = 1)

hate3 = pd.read_csv('english_dataset.tsv',sep='\t')
hate3 = hate3[hate3['task_1'] == 'HOF']
hate3 = hate3.rename({'task_1': 'label'}, axis = 1)

hate4 = pd.read_csv('hasoc2019_en_test-2919.tsv', sep = '\t')
hate4 = hate4[hate4['task_1'] == 'HOF']
hate4 = hate4.rename({'task_1': 'label'}, axis = 1)

hate5 = pd.read_csv('hate5.csv', sep = ';')
hate5.isHate = hate5.isHate.astype(int)
hate5 = hate5[hate5['isHate'] == 1]
hate5 = hate5.rename({'comment': 'text', 'isHate': 'label'}, axis = 1)

train = pd.concat([train, hate1, hate2, hate3, hate4, hate5])
train = train[['text', 'label']]
train.label = train.label.map({0: 0, 1: 1, 'HOF': 1})
print("Class 0 - Class 1: {} - {}".format(len(train[train.label == 0]), len(train[train.label == 1])))

Class 0 - Class 1: 7581 - 6372


In [564]:
train['cleaned_lemma'] = [text_preprocessing(doc, True) for doc in train.text]

,text,label,cleaned_lemma
0,"the scheme saw an estimated 150,000 children f...",1,scheme saw estimated child poor family sent pa...
1,durban 's homeless communities reconciliation ...,1,durban homeless community reconciliation lunch
2,the next immediate problem that cropped up was...,1,next immediate problem cropped assist unfortun...
3,far more important than the implications for t...,1,far important implication economy god dollar l...
4,to strengthen child-sensitive social protectio...,1,strengthen child sensitive social protection s...
...,...,...,...
158,All muslims worship Satan¬ü¬ï¬ã 5 times a day,1,muslim worship satan time day
159,all mental illnesses are awful and must be tre...,1,mental illness awful must treated
160,All girls like you should have a pimp to guide...,1,girl like pimp guide successful prostitution
161,After all this time the african americans are ...,1,time african american people crime rate


In [578]:
vectorizer_tfidf = TfidfVectorizer(max_features = 5000)

X_train_lemma = vectorizer_tfidf.fit_transform(train.cleaned_lemma)
X_valid_lemma = vectorizer_tfidf.transform(valid.cleaned_lemma)

# Train an SPE classifier
lr2 = LogisticRegression(class_weight = 'balanced')
lr2.fit(X_train_lemma, train.label)

# Predict with an SPE classifier
y_pred = lr2.predict(X_valid_lemma)
print(f1_score(lr2.predict(X_train_lemma), train.label))
print(f1_score(y_pred, valid.label))

0.9352062357908413
0.23003194888178913


In [579]:
class_weights = create_class_weight({0: 7581, 1: 6372})

In [580]:
model = Sequential([
Dense(input_dim = 5000, units = 32, activation = 'relu'),
Dense(units = 512, activation = 'relu'),
Dense(units = 256, activation = 'relu'),
Dense(units = 128, activation = 'relu'),
Dense(units = 1, activation = 'sigmoid'),])
model.summary()
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = [get_f1])
model.fit(X_train_lemma.toarray(), train.label, epochs = 20, class_weight = class_weights)
y = model.predict(X_valid_lemma.toarray())
y = y.flatten()
y_pred = np.where(y > 0.5, 1, 0)
f1_score(y_pred, valid.label)

Model: "sequential_91"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_319 (Dense)           (None, 32)                160032    
                                                                 
 dense_320 (Dense)           (None, 512)               16896     
                                                                 
 dense_321 (Dense)           (None, 256)               131328    
                                                                 
 dense_322 (Dense)           (None, 128)               32896     
                                                                 
 dense_323 (Dense)           (None, 1)                 129       
                                                                 
Total params: 341,281
Trainable params: 341,281
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
437/437 [===========================

0.21553884711779447